In [21]:
import pandas as pd

icd_with_summary = pd.read_parquet('../data/processed/structured_dataset_with_discharge_summaries.train.parquet')

icd_with_summary.head()

,subject_id_x,hadm_id,admittime,dischtime,admission_type,discharge_location,gender,dod,age_at_admit,length_of_stay_days,...,num_procedures_total,meds_discharge_like,medication_count,subject_id_y,policy,discharge_summary,diagnosis_codes,model,true_icd_codes,missing_codes
0,17504528,20171885,2137-06-17 20:40:00,2137-06-20 16:41:00,DIRECT EMER.,HOME,F,2142-07-24,56,2.834028,...,1.0,"[5% Dextrose, Albuterol 0.083% Neb Soln, Aspir...",24.0,17504528,partial,History of Present Illness: 56-year-old female...,"[I10, Z8572, Z5181, I5032, I2510, K219, I071, ...",gpt-5-mini,"[I5032, E118, I10, E785, I2510, I071, D649, I2...","[I480, D473, Z8673, Z7901, E785, E118, Z954, F..."
1,14273001,20371042,2187-10-19 19:00:00,2187-10-21 18:13:00,OBSERVATION ADMIT,HOME,F,2188-03-27,73,1.967361,...,NaN,"[Acetaminophen, Aspirin, Atorvastatin, Bisacod...",19.0,14273001,all,History of Present Illness: 73-year-old female...,"[I10, Z87891, C3431, I5031, Y929, T380X5A, K21...",gpt-5-mini,"[I5031, I82501, C7931, C3431, E039, Z7901, I10...",[]
2,11357031,27612249,2139-01-17 21:04:00,2139-01-22 18:00:00,OBSERVATION ADMIT,HOME HEALTH CARE,M,2144-10-28,58,4.872222,...,NaN,"[0.9% Sodium Chloride (Mini Bag Plus), Amoxici...",30.0,11357031,primary_only,History of present illness: 58-year-old man ad...,[I5033],gpt-5-mini,"[I5033, J9692, J9691, E870, E872, E662, J441, ...","[E662, E1165, Z6833, R000, J9692, E785, I272, ..."
3,13673554,25741865,2176-02-25 19:39:00,2176-03-02 17:30:00,OBSERVATION ADMIT,HOME HEALTH CARE,M,2180-11-21,71,5.910417,...,1.0,"[Acetaminophen, Aspirin, Atorvastatin, Cepacol...",17.0,13673554,partial,History of Present Illness: 71-year-old male w...,"[N184, I5033, E1022, J918, Z9641, I739, I2510,...",gpt-5-mini,"[I5033, C9110, J918, N184, N179, I129, I2510, ...",[]
4,19017808,20589756,2183-01-05 21:59:00,2183-01-08 18:35:00,OBSERVATION ADMIT,HOME HEALTH CARE,F,2184-11-16,77,2.858333,...,NaN,"[0.9% Sodium Chloride (Mini Bag Plus), Acetami...",25.0,19017808,partial,Brief history of present illness: 77-year-old ...,"[W19XXXA, K7460, B964, Y929, I5032, E119, N288...",gpt-5-mini,"[I5032, I8510, N179, I272, K7460, N390, I4891,...","[S5001XA, K7581, I10, I4891, N390, I878, N179,..."


In [33]:
icd_with_summary.head(20)

,subject_id_x,hadm_id,admittime,dischtime,admission_type,discharge_location,gender,dod,age_at_admit,length_of_stay_days,...,num_procedures_total,meds_discharge_like,medication_count,subject_id_y,policy,discharge_summary,diagnosis_codes,model,true_icd_codes,missing_codes
0,17504528,20171885,2137-06-17 20:40:00,2137-06-20 16:41:00,DIRECT EMER.,HOME,F,2142-07-24,56,2.834028,...,1.0,"[5% Dextrose, Albuterol 0.083% Neb Soln, Aspir...",24.0,17504528,partial,History of Present Illness: 56-year-old female...,"[I10, Z8572, Z5181, I5032, I2510, K219, I071, ...",gpt-5-mini,"[I5032, E118, I10, E785, I2510, I071, D649, I2...","[I480, D473, Z8673, Z7901, E785, E118, Z954, F..."
1,14273001,20371042,2187-10-19 19:00:00,2187-10-21 18:13:00,OBSERVATION ADMIT,HOME,F,2188-03-27,73,1.967361,...,NaN,"[Acetaminophen, Aspirin, Atorvastatin, Bisacod...",19.0,14273001,all,History of Present Illness: 73-year-old female...,"[I10, Z87891, C3431, I5031, Y929, T380X5A, K21...",gpt-5-mini,"[I5031, I82501, C7931, C3431, E039, Z7901, I10...",[]
2,11357031,27612249,2139-01-17 21:04:00,2139-01-22 18:00:00,OBSERVATION ADMIT,HOME HEALTH CARE,M,2144-10-28,58,4.872222,...,NaN,"[0.9% Sodium Chloride (Mini Bag Plus), Amoxici...",30.0,11357031,primary_only,History of present illness: 58-year-old man ad...,[I5033],gpt-5-mini,"[I5033, J9692, J9691, E870, E872, E662, J441, ...","[E662, E1165, Z6833, R000, J9692, E785, I272, ..."
3,13673554,25741865,2176-02-25 19:39:00,2176-03-02 17:30:00,OBSERVATION ADMIT,HOME HEALTH CARE,M,2180-11-21,71,5.910417,...,1.0,"[Acetaminophen, Aspirin, Atorvastatin, Cepacol...",17.0,13673554,partial,History of Present Illness: 71-year-old male w...,"[N184, I5033, E1022, J918, Z9641, I739, I2510,...",gpt-5-mini,"[I5033, C9110, J918, N184, N179, I129, I2510, ...",[]
4,19017808,20589756,2183-01-05 21:59:00,2183-01-08 18:35:00,OBSERVATION ADMIT,HOME HEALTH CARE,F,2184-11-16,77,2.858333,...,NaN,"[0.9% Sodium Chloride (Mini Bag Plus), Acetami...",25.0,19017808,partial,Brief history of present illness: 77-year-old ...,"[W19XXXA, K7460, B964, Y929, I5032, E119, N288...",gpt-5-mini,"[I5032, I8510, N179, I272, K7460, N390, I4891,...","[S5001XA, K7581, I10, I4891, N390, I878, N179,..."
5,19973083,20741363,2123-10-12 04:00:00,2123-10-20 16:35:00,OBSERVATION ADMIT,HOME HEALTH CARE,F,NaT,58,8.524306,...,NaN,"[Acetaminophen, Amiodarone, Aspirin EC, Atorva...",12.0,19973083,primary_only,58-year-old female admitted under observation ...,[I5033],gpt-5-mini,"[I5033, N170, J9691, D6861, I4892, I340, Z7901...","[Z90710, I4892, I480, Z8673, I440, R351, N170,..."
6,14541392,25068197,2124-09-02 00:07:00,2124-09-03 15:36:00,EW EMER.,HOME,M,2125-10-01,58,1.645139,...,NaN,"[Acetaminophen, Albuterol 0.083% Neb Soln, Bis...",25.0,14541392,primary_only,History of present illness: 58-year-old male p...,[I5033],gpt-5-mini,"[I5033, G92, J441, I8510, Z6842, F17210, K7469...","[E669, F1190, K7469, R0789, Y92009, K7030, M79..."
7,10320090,26774424,2178-04-05 00:00:00,2178-04-12 15:46:00,URGENT,HOME,M,2179-05-25,33,7.656944,...,NaN,"[5% Dextrose, Acetaminophen, Amiodarone, Clobe...",17.0,10320090,primary_only,Chief complaint: Worsening dyspnea and volume ...,[I5023],gpt-5-mini,"[I5023, I429, N179, Z6843, E669, I480, D638, L...","[E669, Z23, Z86718, I480, I2510, L409, N179, Z..."
8,10906285,29225762,2179-09-17 17:46:00,2179-09-23 16:12:00,OBSERVATION ADMIT,HOME HEALTH CARE,M,NaT,70,5.934722,...,NaN,"[Acetaminophen, Atorvastatin, BuPROPion XL (On...",16.0,10906285,partial,70-year-old male admitted under observation fo...,"[Z20822, I5033, D539, N400, Z86718, K219, F32A...",gpt-5-mini,"[I5033, C8510, I428, N400, F32A, K219, I2584, ...","[E669, D696, I2584, Z87891, C8510, Z7901, I428]"
9,15559495,21036461,2116-08-10 22:43:00,2116-08-28 15:37:00,OBSERVATION ADMIT,HOME HEALTH CARE,F,NaT,78,17.704167,...,3.0,"[Acetaminophen, Apixaban, Bisacodyl, Bumetanid...",16.0,15559495,all,History of present illness: 78-year-old female...,"[I081, G4733, K761, T84092A, R0902, D509, R943...",gpt-5-mini,

In [38]:
# want to write a sample summary text to a text file
with open('sample_summary.txt', 'w') as f:
  f.write(icd_with_summary.iloc[19]['discharge_summary'])


In [34]:
sorted(icd_with_summary.iloc[19]['diagnosis_codes'])

['E1140',
 'E1151',
 'E785',
 'I480',
 'I70212',
 'T82858A',
 'Z794',
 'Z87891',
 'Z952']

In [35]:
sorted(icd_with_summary.iloc[19]['missing_codes'])

['E1121', 'Z7901', 'Z89421']

In [37]:
sorted(icd_with_summary.iloc[19]['true_icd_codes'])

['E1121',
 'E1140',
 'E1151',
 'E785',
 'I480',
 'I70212',
 'T82858A',
 'Z7901',
 'Z794',
 'Z87891',
 'Z89421',
 'Z952']

In [7]:
icd_with_summary.shape

(1046, 27)

In [2]:
'''
embed the summaries

id in the chroma collection id hadm_id
store all the true icd codes so that we can see what is associated with the summary
metadata structure
{
  'icd_codes': ['abc123'],
  'discharge_summary': str
}
'''

from pydantic import BaseModel, Field
import uuid

class DischargeSummaryEmbedding(BaseModel):
  icd_codes: str = Field(..., description = 'A comma separated string of the true icd codes assigned to the summary')
  discharge_summary: str = Field(..., description = 'The text of the discharge summary')

In [3]:

docs = []
metadatas = []
# embeddings = []
ids = []
for i, row in icd_with_summary.iterrows():
  summary = row['discharge_summary']
  icd_codes = ",".join(row['true_icd_codes'])
  id = str(row['hadm_id'])
  doc = DischargeSummaryEmbedding(
    icd_codes = icd_codes,
    discharge_summary = summary
  )
  docs.append(summary)
  metadatas.append(doc.model_dump())
  ids.append(id)

In [14]:
metadatas[7]

{'icd_codes': 'I5023,I429,N179,Z6843,E669,I480,D638,L409,G8929,M25569,Z9114,Z23,N189,I2510,I340,Z95810,Z86718,Z7982,Z4502,T501X6A',
 'discharge_summary': '**Patient Information:** 33-year-old male, admitted urgently with acute on chronic systolic (congestive) heart failure (ICD10: I5023). \n\n**History of Present Illness:** The patient presented with increased dyspnea on exertion, orthopnea, and lower extremity edema. He has a history of congestive heart failure and obesity (BMI 50.0-59.9), with a recent exacerbation likely triggered by non-compliance with medications and dietary restrictions.\n\n**Hospital Course:** During a 7.7-day hospital stay, the patient was stabilized with diuretics (Furosemide, Spironolactone) to manage fluid overload. Amiodarone was initiated for paroxysmal atrial fibrillation, and Lisinopril was continued to manage hypertension and heart failure. The patient underwent daily assessments for fluid balance and renal function, which showed improvement with decrea

In [4]:
from chromadb.utils import embedding_functions
from chromadb.config import Settings
import chromadb

sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name = "lokeshch19/ModernPubMedBERT")

chroma_client = chromadb.PersistentClient(path="../patient_summary_embeddings", settings=Settings(allow_reset=True))

chroma_collection = chroma_client.get_or_create_collection("patient_summary_embeddings", embedding_function=sentence_transformer_ef)

In [ ]:
chroma_collection

In [5]:
chroma_collection.upsert(
    ids=ids,
    documents=docs,
    metadatas=metadatas
)

In [3]:
# test some validation queries


val_data = pd.read_parquet('../data/processed/structured_dataset_with_discharge_summaries.val.parquet')

In [4]:
val_data.head()

,subject_id,hadm_id,admittime,dischtime,admission_type,discharge_location,gender,dod,age_at_admit,length_of_stay_days,...,procedures_icd10,num_procedures_total,meds_discharge_like,medication_count,policy,discharge_summary,diagnosis_codes,model,true_icd_codes,missing_codes
0,13917492,27765450,2114-01-16 01:58:00,2114-02-04 16:50:00,EW EMER.,HOME HEALTH CARE,M,2114-02-18,72,19.619444,...,None,NaN,"[5% Dextrose, Acetaminophen, Apixaban, Atorvas...",18.0,partial,History of Present Illness: 72-year-old man ad...,"[Z515, I428, Q6432, Z955, R570, I2510, I5023, ...",gpt-5-mini,"[I5023, N170, I21A1, R570, K7200, E43, I4821, ...","[I4821, I21A1, I255, E861, Z6822, K7200, E43, ..."
1,17302284,22205215,2164-02-03 16:48:00,2164-02-08 16:15:00,OBSERVATION ADMIT,HOME,M,2170-12-14,67,4.977083,...,None,NaN,"[5% Dextrose, Amiodarone, Aspirin, Carvedilol,...",15.0,all,History of Present Illness: 67-year-old man ad...,"[N189, E785, Z955, I4891, I2510, I255, Z951, I...",gpt-5-mini,"[I5023, I272, I4891, I255, I340, I2510, Z951, ...",[]
2,17240046,25129064,2178-11-19 15:21:00,2178-11-23 16:05:00,OBSERVATION ADMIT,HOME,M,2180-10-10,61,4.030556,...,"[047K3ZZ, 047M3ZZ, 0Y6T0Z3, 5A1D70Z]",4.0,"[Acetaminophen, Aspirin, Atorvastatin, Calcium...",35.0,primary_only,History of Present Illness: 61-year-old male w...,[E1152],gpt-5-mini,"[E1152, N186, I70261, I120, E1122, D631, G546,...","[G546, Z951, I120, K219, Z89612, Z992, D631, I..."
3,15346570,27869814,2164-11-05 17:11:00,2164-11-12 12:44:00,OBSERVATION ADMIT,HOME HEALTH CARE,M,NaT,75,6.814583,...,"[0Y6P0Z1, B41FYZZ]",2.0,"[Acetaminophen, Apixaban, Aspirin, Bisacodyl, ...",27.0,partial,History of Present Illness: 75-year-old man ad...,"[L97512, E11621, Z7902, M86171, I2510, E1152, ...",gpt-5-mini,"[E1169, E1152, M86171, I70261, Z7902, I10, I25...",[I10]
4,16273050,26152108,2151-06-11 17:44:00,2151-06-21 12:46:00,EW EMER.,HOME HEALTH CARE,F,NaT,33,9.793056,...,None,NaN,"[5% Dextrose, Acetaminophen, Bisacodyl, DULoxe...",24.0,partial,Brief history of present illness: 33-year-old ...,"[E1143, I10, Z794, E1165, K224]",gpt-5-mini,"[E1143, E1165, I10, E441, M797, K3184, Z794, Z...","[D509, E1140, E441, K219, M797, E669, Z6839, Z..."


In [9]:
for i, row in val_data[:5].iterrows():
  print(pd.isna(row['diagnosis_codes'][0]))

False
False
False
False
False


In [5]:
val_data.columns

Index(['subject_id', 'hadm_id', 'admittime', 'dischtime', 'admission_type',
       'discharge_location', 'gender', 'dod', 'age_at_admit',
       'length_of_stay_days', 'icd10_codes', 'icd10_descriptions',
       'num_icd10_codes', 'primary_icd10', 'icd_version', 'primary_icd10_desc',
       'primary_pdgm_bucket_simple', 'procedures_icd10',
       'num_procedures_total', 'meds_discharge_like', 'medication_count',
       'policy', 'discharge_summary', 'diagnosis_codes', 'model',
       'true_icd_codes', 'missing_codes'],
      dtype='object')

In [8]:
val_data['icd10_codes'].apply(lambda x: x.tolist()) == val_data['true_icd_codes'].apply(lambda x: x.tolist())

0      True
1      True
2      True
3      True
4      True
       ... 
218    True
219    True
220    True
221    True
222    True
Length: 223, dtype: bool

In [8]:
summaries = val_data['discharge_summary'].iloc[:5]
codes = val_data['true_icd_codes'].iloc[:5]

results = chroma_collection.query(
    query_texts = summaries.tolist(),
    # n_results = 5
)

In [ ]:
results['metadatas'][0][-1]

[{'icd_codes': 'I130,I5023,I21A1,N179,N184,I050,Z952,I2510,Z951,E1121,J449,E785,G2581,K219,Z794,F419,F329,Z87891',
  'discharge_summary': 'History of present illness: 67-year-old man with a history of prosthetic mitral valve, prior CABG, ischemic heart disease, type 2 diabetes with nephropathy, COPD and known chronic kidney disease (stage 4) was admitted urgently for acute decompensated systolic heart failure with volume overload and concurrent acute kidney injury. Troponin elevation consistent with type 2 myocardial infarction was also noted during admission.\n\nHospital course: Over a 9.7-day hospitalization the patient received guided IV diuresis with careful electrolyte and renal monitoring, afterload reduction with hydralazine and isosorbide mononitrate for symptomatic ischemia and blood pressure control, optimization of glycemic control with insulin, and inhaled bronchodilator therapy for COPD exacerbation. Serial cardiac enzymes and telemetry were monitored; no ischemic interven

In [25]:
results.keys()

dict_keys(['ids', 'embeddings', 'documents', 'uris', 'included', 'data', 'metadatas', 'distances'])

In [10]:
results['distances'][0]

[0.11317634582519531,
 0.12990045547485352,
 0.13044601678848267,
 0.13913506269454956,
 0.13949400186538696,
 0.140264630317688,
 0.14460068941116333,
 0.15482497215270996,
 0.16387319564819336,
 0.16665416955947876]

In [11]:
print(results['documents'][0][-1])

History of Present Illness: 64-year-old male admitted under observation for acute on chronic heart failure in the setting of hypertensive heart disease and chronic kidney disease (CKD stage 4). He presented with progressive dyspnea, lower extremity edema, and poorly controlled blood pressure. Significant comorbidities include atrial septal defect, type 2 diabetes mellitus with diabetic CKD, COPD/OSA, prior colorectal malignancy, hyperlipidemia, and anemia of CKD.

Hospital Course: Length of stay 7.2 days. Patient received diuretic optimization with symptomatic improvement in congestion, careful volume management given CKD stage 4, and blood pressure control with continuation/adjustment of carvedilol and addition/titration of hydralazine as indicated. Glycemic management was achieved with basal-bolus insulin while inpatient. Inhaled bronchodilator and inhaled corticosteroid/long-acting beta-agonist therapy were continued for COPD. Antiplatelet therapy (aspirin) was continued; prophylact

In [17]:
sorted(results['metadatas'][0][-1]['icd_codes'].split(","))

['C182',
 'D122',
 'D123',
 'D509',
 'D631',
 'E039',
 'E1122',
 'E1165',
 'E559',
 'E785',
 'F329',
 'G3184',
 'G4733',
 'I130',
 'I2510',
 'I5033',
 'I69922',
 'J449',
 'K219',
 'M109',
 'M1990',
 'N184',
 'Q211',
 'Z20822',
 'Z794',
 'Z87891',
 'Z9111',
 'Z9181']

In [12]:
print(summaries[0])

History of Present Illness: 72-year-old man admitted emergently with acute on chronic systolic heart failure (I50.23) complicated by cardiogenic shock and type 2 myocardial infarction. Admission course was notable for acute tubular necrosis (AKI, N17.0), severe protein-calorie malnutrition, hyponatremia, and hepatic dysfunction. Prior history includes permanent atrial fibrillation and chronic ischemic heart disease.

Hospital Course: Patient required hemodynamic support with dobutamine infusion for cardiogenic shock and continuous telemetry. Over a 19.6-day hospitalization he was diuresed judiciously, received careful electrolyte and volume management, and required temporary renal support measures while AKI improved but did not fully normalize. Nutrition was optimized with dietitian involvement and oral nutrition supplements for severe malnutrition. Hyponatremia and hepatic chemistries trended toward baseline with supportive care. Antithrombotic strategy was implemented given atrial fi

In [19]:
sorted(codes[0])

['E43',
 'E861',
 'E871',
 'E872',
 'F4310',
 'I21A1',
 'I2510',
 'I255',
 'I340',
 'I428',
 'I4821',
 'I5023',
 'K5900',
 'K7200',
 'N170',
 'Q6432',
 'R570',
 'Z20822',
 'Z515',
 'Z66',
 'Z6822',
 'Z7901',
 'Z955']

In [15]:
icd_10_data = pd.read_csv('/Users/benjamindykstra/development/icd-10-coding/server/data/icd10_links/icd10_code_descriptions.csv')

In [16]:
icd_10_data.head()

,href,url,error,title,code_x,breadcrumbs,code_description,meta_description,badges,billable,poa_exempt,applicable_to,approximate_synonyms,present_on_admission,clinical_information,code_y
0,/ICD10CM/Codes/A00-B99/A00-A09/A02-/A02.1,https://www.icd10data.com/ICD10CM/Codes/A00-B9...,NaN,2026 ICD-10-CM Diagnosis Code A02.1,A02.1,ICD-10-CM Codes > A00-B99 > A00-A09 > A02- > 2...,Salmonella sepsis,ICD 10 code for Salmonella sepsis. Get free ru...,2016\n2017\n2018\n2019\n2020\n2021\n2022\n2023...,True,False,Septicemia NOS,Salmonella septicemia\nSalmonella septicemia w...,NaN,NaN,A021
1,/ICD10CM/Codes/A00-B99/A00-A09/A04-/A04.7,https://www.icd10data.com/ICD10CM/Codes/A00-B9...,NaN,2026 ICD-10-CM Diagnosis Code A04.7,A04.7,ICD-10-CM Codes > A00-B99 > A00-A09 > A04- > 2...,Enterocolitis due to Clostridium difficile,ICD 10 code for Enterocolitis due to Clostridi...,2016\n2017\n2018 - Converted to Parent Code\n2...,False,False,Clostridioides difficile colitis\nFoodborne in...,C difficile diarrhea\nClostridium difficile co...,NaN,NaN,A047
2,/ICD10CM/Codes/A00-B99/A00-A09/A04-/A04.4,https://www.icd10data.com/ICD10CM/Codes/A00-B9...,NaN,2026 ICD-10-CM Diagnosis Code A04.4,A04.4,ICD-10-CM Codes > A00-B99 > A00-A09 > A04- > 2...,Other intestinal Escherichia coli infections,ICD 10 code for Other intestinal Escherichia c...,2016\n2017\n2018\n2019\n2020\n2021\n2022\n2023...,True,False,Escherichia coli enteritis NOS,E coli enteritis\nIntestinal infection due to ...,NaN,NaN,A044
3,/ICD10CM/Codes/A00-B99/A00-A09/A04-/A04.71,https://www.icd10data.com/ICD10CM/Codes/A00-B9...,NaN,2026 ICD-10-CM Diagnosis Code A04.71,A04.71,ICD-10-CM Codes > A00-B99 > A00-A09 > A04- > 2...,"Enterocolitis due to Clostridium difficile, re...",ICD 10 code for Enterocolitis due to Clostridi...,2018 - New Code\n2019\n2020\n2021\n2022\n2023\...,True,False,Clostridioides difficile colitis\nFoodborne in...,NaN,NaN,NaN,A0471
4,/ICD10CM/Codes/A00-B99/A00-A09/A02-/A02.0,https://www.icd10data.com/ICD10CM/Codes/A00-B9...,NaN,2026 ICD-10-CM Diagnosis Code A02.0,A02.0,ICD-10-CM Codes > A00-B99 > A00-A09 > A02- > 2...,Salmonella enteritis,ICD 10 code for Salmonella enteritis. Get free...,2016\n2017\n2018\n2019\n2020\n2021\n2022\n2023...,True,False,Salmonellosis,Salmonella gastroenteritis,NaN,Poisoning caused by ingestion of food harborin...,A020


In [17]:
sliced = icd_10_data[['code_x', 'code_description']].rename(columns={'code_x': 'code', 'code_description': 'description'})


In [18]:
sliced.head()

,code,description
0,A02.1,Salmonella sepsis
1,A04.7,Enterocolitis due to Clostridium difficile
2,A04.4,Other intestinal Escherichia coli infections
3,A04.71,"Enterocolitis due to Clostridium difficile, re..."
4,A02.0,Salmonella enteritis


In [20]:
import json

# write this to a json file
with open('icd10_codes.json', 'w') as f:
  json.dump(sliced.to_dict(orient='records'), f)


In [40]:
validation_results = pd.read_csv('../results/validation/api_validation_results.csv')

In [41]:
validation_results.head()

,idx,hadm_id,num_provided_codes,num_true_missing,num_api_missing,elapsed_time,true_positives,false_positives,false_negatives,precision,recall,f1,num_api_codes,num_true_codes
0,0,27765450,11,12,5,3.374886,[],"['E460', 'E870', 'I214', 'R7989', 'Z721']","['E43', 'E861', 'E871', 'E872', 'I21A1', 'I255...",0.000000,0.000000,0.000000,5,12
1,1,22205215,18,0,5,6.501931,[],"['I471', 'I4819', 'R609', 'Z5183', 'Z723']",[],0.000000,0.000000,0.000000,5,0
2,2,25129064,1,16,9,7.936413,"['D631', 'E785', 'I2510', 'I480', 'N186', 'Z79...",['I739'],"['E1122', 'G546', 'I120', 'I70261', 'K219', 'Z...",0.888889,0.500000,0.640000,9,16
3,3,27869814,9,1,3,6.002349,[],"['I739', 'M89671', 'Z6843']",['I10'],0.000000,0.000000,0.000000,3,1
4,4,26152108,5,13,5,4.028611,"['E669', 'K3184', 'M797']","['E11629', 'E46']","['D509', 'E11319', 'E1140', 'E441', 'E785', 'K...",0.600000,0.230769,0.333333,5,13


In [43]:
validation_results[['precision', 'recall', 'f1']].describe()

,precision,recall,f1
count,223.000000,223.000000,223.000000
mean,0.303281,0.214738,0.233173
std,0.296607,0.246538,0.238029
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.285714,0.153846,0.210526
75%,0.500000,0.333333,0.400000
max,1.000000,1.000000,1.000000


In [44]:
val_rag_results = pd.read_csv('../results/validation/api_validation_results_with_rag.csv')
val_rag_results.head()
val_rag_results[['precision', 'recall', 'f1']].describe()


,precision,recall,f1
count,70.000000,70.000000,70.000000
mean,0.240385,0.145178,0.163890
std,0.279677,0.203947,0.198088
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.166667,0.087121,0.121324
75%,0.400000,0.196875,0.265217
max,1.000000,1.000000,0.823529


In [45]:
set('abc')

{'a', 'b', 'c'}

In [ ]:
# sort list of tuples by first and then second eleemnt ascending
[(2, 'a'), (1, 'b'), (2, 'f')].sort(key=lambda x: (x[0], x[1])) # => [(2, 'a'), (2, 'f'), (1, 'b')]



In [ ]:
from utils.vector_store_utils import vector_store